In [1]:
import streamlit as st
import plotly.graph_objects as go
from datetime import datetime, timedelta
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [3]:
# Set dark theme
st.set_page_config(
    page_title="Stock Price Prediction",
    layout="wide",
    initial_sidebar_state="collapsed"
)

In [5]:
# Custom CSS for dark theme
st.markdown("""
<style>
.stApp {
    background-color: #202124;
    color: #ffffff;
}
.stock-header {
    font-size: 2.5em;
    font-weight: bold;
    margin-bottom: 0.5em;
}
.stock-price {
    font-size: 2em;
    font-weight: bold;
}
.price-change {
    color: #ff4444;
    font-size: 1.2em;
}
.metric-card {
    background-color: #303134;
    padding: 1em;
    border-radius: 10px;
    margin: 0.5em;
}
.tab-container {
    display: flex;
    gap: 1em;
    margin-bottom: 1em;
}
.tab {
    background-color: #303134;
    padding: 0.5em 2em;
    border-radius: 20px;
    cursor: pointer;
}
.tab.active {
    background-color: #8ab4f8;
    color: #202124;
}
</style>
""", unsafe_allow_html=True)


2024-11-16 04:35:04.060 
  command:

    streamlit run D:\anaconda\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [7]:
# Function to fetch stock data
def fetch_stock_data(symbol, start_date, end_date):
    stock = yf.Ticker(symbol)
    df = stock.history(start=start_date, end=end_date)
    return df[['Open', 'High', 'Low', 'Close', 'Volume']]


In [9]:
# Function to prepare stock data for LSTM model
def prepare_stock_data(symbol):
    # Fetch data for the past 5 years
    end_date = datetime.today().date()
    start_date = end_date - timedelta(days=5*365)
    df = fetch_stock_data(symbol, start_date, end_date)
    
    # Normalize the data
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df[['Open', 'High', 'Low', 'Close', 'Volume']])
    
    return df, scaled_data, scaler

In [11]:
# Function to create LSTM model
def create_lstm_model(data, time_steps, future_days):
    # Normalize the data to a range between 0 and 1
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data)
    
    # Prepare the data for LSTM with the given time steps
    X, y = [], []
    for i in range(time_steps, len(scaled_data) - future_days + 1):
        X.append(scaled_data[i - time_steps:i])
        y.append(scaled_data[i:i + future_days, 3])  # '3' is for 'Close' column index
    
    # Convert to numpy arrays for model training
    X, y = np.array(X), np.array(y)
    n_features = X.shape[2]
    
    # Define the LSTM model structure
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(time_steps, n_features)),
        LSTM(50, return_sequences=False),
        Dense(future_days)
    ])
    model.compile(optimizer='adam', loss='mse')
    
    # Train the model
    model.fit(X, y, epochs=50, batch_size=32, validation_split=0.1, verbose=1)
    
    return model, scaler

In [13]:
# Function to predict future prices
def predict_future_prices(model, scaler, data, time_steps, future_days):
    scaled_data = scaler.transform(data[['Open', 'High', 'Low', 'Close', 'Volume']])
    X_pred = []
    for i in range(len(scaled_data) - time_steps, len(scaled_data)):
        X_pred.append(scaled_data[i - time_steps:i])
    
    X_pred = np.array(X_pred)
    predictions = model.predict(X_pred)
    
    # Inverse transform the predictions
    predicted_scaled = scaler.inverse_transform(np.column_stack((predictions, np.zeros((future_days, 4)))))
    predicted_prices = predicted_scaled[:, 3]
    
    return predicted_prices

In [15]:
# Streamlit app
st.title('Stock Market Prediction App')

DeltaGenerator()

In [17]:
# Input for stock symbol
symbol = st.text_input("Enter Stock Symbol (e.g., AAPL, MSFT, AMZN):")


2024-11-16 04:36:17.729 Session state does not function when running a script without `streamlit run`


In [19]:
# Common stock symbols
common_symbols = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'FB', 'TSLA', 'NVDA', 'JPM', 'JNJ', 'V']

In [21]:
if symbol:
    try:
        # Check if the symbol needs a suffix
        if '.' not in symbol and symbol.isalpha():
            symbol = f"{symbol}"  # Add .NS suffix for Indian stocks

        # Fetch current stock data
        stock = yf.Ticker(symbol)
        current_data = stock.history(period='1d')

        if current_data.empty:
            st.error(f"No data found for symbol {symbol}. Please check the symbol and try again.")
            st.info(f"Here are some common stock symbols you can try: {', '.join(common_symbols)}")
        else:
            current_price = current_data['Close'].iloc[-1]
            price_change = current_data['Close'].iloc[-1] - current_data['Open'].iloc[0]
            price_change_percent = (price_change / current_data['Open'].iloc[0]) * 100

            # Display stock header and current price
            st.markdown(f"<div class='stock-header'>{stock.info.get('longName', symbol)}</div>", unsafe_allow_html=True)
            st.markdown(f"<div class='stock-price'>{current_price:.2f} {stock.info.get('currency', 'USD')}</div>", unsafe_allow_html=True)
            st.markdown(f"<div class='price-change'>{price_change:.2f} ({price_change_percent:.2f}%) today</div>", unsafe_allow_html=True)

            # Time period selector
            periods = ['1D', '5D', '1M', '6M', 'YTD', 'More']
            selected_period = st.select_slider('Select Time Period', options=periods, value='1D')

            # Fetch and display stock data based on selected period
            if selected_period == '1D':
                historical_data = stock.history(interval='5m', period='1d')
            elif selected_period == '5D':
                historical_data = stock.history(interval='15m', period='5d')
            elif selected_period == '1M':
                historical_data = stock.history(interval='1d', period='1mo')
            elif selected_period == '6M':
                historical_data = stock.history(interval='1d', period='6mo')
            else:
                historical_data = stock.history(interval='1d', period='ytd')

            # Create price chart
            fig = go.Figure()
            fig.add_trace(go.Scatter(
                x=historical_data.index,
                y=historical_data['Close'],
                mode='lines',
                name='Price',
                line=dict(color='blue')
            ))
            st.plotly_chart(fig, use_container_width=True)

            # Prediction button
            if st.button('Show Price Prediction'):
                st.write("Making Predictions...")
                df, scaled_data, scaler = prepare_stock_data(symbol)
                model, scaler = create_lstm_model(df, time_steps=60, future_days=7)
                predictions = predict_future_prices(model, scaler, df, time_steps=60, future_days=7)
                
                st.write("Predicted Prices for the Next 7 Days:")
                st.write(predictions)

    except Exception as e:
        st.error(f"Error fetching data for {symbol}: {str(e)}")
else:
    st.info("Please enter a stock symbol to start.")